In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
batchsize=8

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2, validation_split=0.2)

In [4]:
train_data= train_datagen.flow_from_directory(r"C:\Users\golla\drowsiness\Prepared_Data\train",
                          target_size=(80,80),batch_size=batchsize,class_mode="categorical", subset="training")
validation_data= train_datagen.flow_from_directory(r"C:\Users\golla\drowsiness\Prepared_Data\train",
                          target_size=(80,80),batch_size=batchsize,class_mode="categorical", subset="validation")                                          

Found 23058 images belonging to 2 classes.
Found 5763 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r"C:\Users\golla\drowsiness\Prepared_Data\test",
                          target_size=(80,80),batch_size=batchsize,class_mode="categorical")


Found 4573 images belonging to 2 classes.


In [6]:
bmodel = MobileNet(include_top=False, weights="imagenet",input_shape=(87,87,3))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation="relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation="sigmoid")(hmodel)

model = Model(inputs=bmodel.input,outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [8]:
checkpoint = ModelCheckpoint(r"C:\Users\golla\drowsiness\models\model.h5",monitor="val_loss",save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = "val_loss",patience=7,verbose=3, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(moniter= "val_loss",patience=3,verbose= 3, )
callbacks =[checkpoint,earlystop,learning_rate]

In [9]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              #loss=tf.keras.losses.BinaryCrossentropy(),
             # metrics=[tf.keras.metrics.BinaryAccuracy(),
                       #tf.keras.metrics.FalseNegatives()])
#model.fit(train_data,batch_size=batchsize,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=callbacks,epochs=10)
#model.fit(train_data,epochs=1,verbose=1,shuffle=True,class_weight=None,sample_weight=None,initial_epoch=0,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=None,validation_batch_size=None,validation_freq=1,max_queue_size=10,workers=1,use_multiprocessing=False)
#model.fit(tf.expand_dims(train_data,axis=-1),batch_size=batchsize,epochs=10,verbose=1,validation_data=(test_data))

model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)



Epoch 1/5
2882/2882 [==============================] - ETA: 0s - loss: 0.2339 - accuracy: 0.9095
Epoch 1: val_loss improved from inf to 0.44729, saving model to C:\Users\golla\drowsiness\models\model.h5
2882/2882 [==============================] - 271s 94ms/step - loss: 0.2339 - accuracy: 0.9095 - val_loss: 0.4473 - val_accuracy: 0.8594 - lr: 0.0010
Epoch 2/5
2882/2882 [==============================] - ETA: 0s - loss: 0.1831 - accuracy: 0.9303
Epoch 2: val_loss improved from 0.44729 to 0.31410, saving model to C:\Users\golla\drowsiness\models\model.h5
2882/2882 [==============================] - 119s 41ms/step - loss: 0.1831 - accuracy: 0.9303 - val_loss: 0.3141 - val_accuracy: 0.8806 - lr: 0.0010
Epoch 3/5
2881/2882 [============================>.] - ETA: 0s - loss: 0.1720 - accuracy: 0.9324
Epoch 3: val_loss did not improve from 0.31410
2882/2882 [==============================] - 106s 37ms/step - loss: 0.1720 - accuracy: 0.9325 - val_loss: 0.7575 - val_accuracy: 0.8196 - lr: 0.0010

In [10]:
# model Evaluation

In [11]:
acc_tr , loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\golla\AppData\Local\Temp/ipykernel_25084/2876905264.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr , loss_tr = model.evaluate_generator(train_data)


0.12622064352035522
0.9513400793075562


In [12]:
acc_vr , loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\golla\AppData\Local\Temp/ipykernel_25084/3811301195.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr , loss_vr = model.evaluate_generator(validation_data)


0.43460407853126526
0.8877320885658264


In [13]:
acc_test , loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

C:\Users\golla\AppData\Local\Temp/ipykernel_25084/3089708754.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test , loss_test = model.evaluate_generator(test_data)


0.12622064352035522
0.9513400793075562
